In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the path to the data directory
data_dir = "/content/drive/MyDrive/archive"

In [ ]:
# Set the image size
IMG_SIZE = 128

In [ ]:
# Set the batch size
batch_size = 32

In [ ]:
# Create a data generator for training data
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.2
)

In [ ]:
# Create a data generator for test data
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [ ]:
# Load the training data
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size = (128, 128),
    batch_size = 32,
    class_mode = "categorical",
    subset = "training"
)

Found 546 images belonging to 2 classes.


In [ ]:
# Load the validation data
val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size = (128, 128),
    batch_size = 32,
    class_mode = "categorical",
    subset = "validation"
)

Found 135 images belonging to 2 classes.


In [ ]:
# Load the test data
test_data = test_datagen.flow_from_directory(
    data_dir,
    target_size = (128, 128),
    batch_size = 1,
    shuffle = False,
    class_mode = "categorical"
)

Found 681 images belonging to 2 classes.


In [ ]:
# Load the VGG16 model
vgg = VGG16(input_shape = (128,128, 3), weights = "imagenet", include_top = False)

In [ ]:
# Freeze the layers
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# Add a flatten layer
x = Flatten()(vgg.output)

# Add a dense layer with 1024 units and ReLU activation
x = Dense(1024, activation = "relu")(x)

# Add a dropout layer with a rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a dense layer with the number of classes and softmax activation
predictions = Dense(train_data.num_classes, activation = "softmax")(x)

In [ ]:
# Create a model object
model = Model(inputs = vgg.input, outputs = predictions)

In [ ]:
# Compile the model
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
# Print the model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
# Train the model
history = model.fit(
    train_data,
    validation_data = val_data,
    epochs = 10,
    verbose = 1
)

Epoch 1/10
18/18 [==============================] - 202s 11s/step - loss: 1.3400 - accuracy: 0.7582 - val_loss: 0.2044 - val_accuracy: 0.9407
Epoch 2/10
18/18 [==============================] - 166s 9s/step - loss: 0.1368 - accuracy: 0.9432 - val_loss: 0.1530 - val_accuracy: 0.9630
Epoch 3/10
18/18 [==============================] - 171s 9s/step - loss: 0.1163 - accuracy: 0.9579 - val_loss: 0.1201 - val_accuracy: 0.9704
Epoch 4/10
18/18 [==============================] - 169s 9s/step - loss: 0.0802 - accuracy: 0.9707 - val_loss: 0.1040 - val_accuracy: 0.9556
Epoch 5/10
18/18 [==============================] - 169s 9s/step - loss: 0.0615 - accuracy: 0.9853 - val_loss: 0.1259 - val_accuracy: 0.9556
Epoch 6/10
18/18 [==============================] - 161s 9s/step - loss: 0.0766 - accuracy: 0.9762 - val_loss: 0.1914 - val_accuracy: 0.9407
Epoch 7/10
18/18 [==============================] - 171s 10s/step - loss: 0.0942 - accuracy: 0.9689 - val_loss: 0.2268 - val_accuracy: 0.9333
Epoch 8/10


In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_data, verbose = 1)

681/681 [==============================] - 166s 243ms/step - loss: 0.0401 - accuracy: 0.9868


In [ ]:
# Print the test accuracy
print("Test accuracy:", test_acc)

Test accuracy: 0.9867841601371765


In [ ]:
# Save the model
model.save("VGG16.h5")

In [ ]:
from keras.preprocessing import image


In [ ]:
# load the saved model
model = keras.models.load_model('/content/VGG16.h5')

# path to an image that you want to predict on
img_path = '/content/15.jpg'

from PIL import Image
import numpy as np

# load the image and preprocess it
img = Image.open(img_path)
img = img.resize((128, 128))
x = np.array(img)
x = np.expand_dims(x, axis=0)

# make the prediction
preds = model.predict(x)

# get the predicted class label
predicted_class = np.argmax(preds)

# print the predicted class label
if predicted_class == 0:
    print('NORMAL')
else:
    print('POTHOLES')


1/1 [==============================] - 0s 344ms/step
NORMAL
